In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import skimage as sk
from skimage import measure
import os
import tifffile
from tqdm import tqdm

In [2]:
dots_data = pd.read_csv("field_001.gated_dots.tsv", sep="\t")

In [3]:
dots_data2 = dots_data.loc["60x" == dots_data["magnification"], :]

In [4]:
dots_data2

,series_id,scaling,magnification,image_type,Channel,Nuclei,x,y,z,Value,FWHM,SNR,NSNR,Label,SNR2,nid,Value2,selected
0,1,0.261319,60x,dw,a647,0,1090.139,1159.156,26.813,14.171,2.180,1605.275,-1.000,0,950.673138,100,54.228740,True
1,1,0.261319,60x,dw,a647,0,1646.243,1290.971,27.028,14.164,2.242,1673.092,-1.000,0,1832.370135,140,54.201952,True
2,1,0.261319,60x,dw,a647,11,945.030,144.853,26.856,13.368,2.717,inf,90.549,0,50383.326358,84,51.155867,True
3,1,0.261319,60x,dw,a647,0,1313.026,1843.882,26.009,13.224,2.051,inf,-1.000,0,203056.421247,111,50.604816,True
4,1,0.261319,60x,dw,a647,37,1224.944,367.632,25.658,13.055,2.448,1093.883,100.587,0,2613.070001,105,49.958097,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2776,1,1.000000,60x,raw,a647,0,2021.014,1863.917,26.755,0.211,3.903,13.145,-1.000,0,12.471165,177,0.211000,True
2777,1,1.000000,60x,raw,a647,0,175.016,164.047,28.935,0.209,3.207,12.918,-1.000,0,9.685818,18,0.209000,True
2778,1,1.000000,60x,raw,a647,0,1028.118,1261.987,26.917,0.206,3.060,15.004,-1.000,0,15.226245,96,0.206000,True
2779,1,1.000000,60x,raw,a647,13,186.044,142.026,26.032,0.206,2.415,15.374,1.464,0,14.838028,18,0.206000,True


In [5]:
ref_raw = dots_data2.loc["raw" == dots_data2["image_type"], :].reset_index(drop=True)
ref__dw = dots_data2.loc["dw" == dots_data2["image_type"], :].reset_index(drop=True)
raw_image_folder_path = "/mnt/data/Imaging/202105-Deconwolf/data_210726/60x_raw"
dw__image_folder_path = "/mnt/data/Imaging/202105-Deconwolf/data_210726/60x_dw"
mask_folder_path = "../../data/60x_mask/dilated_labels_watershed"

In [6]:
current_field_id = 1

print(f"Field #{current_field_id}")

raw_max_z_proj = tifffile.imread(os.path.join(raw_image_folder_path, f"a647_{current_field_id:03d}.tif")).max(0)
dw__max_z_proj = tifffile.imread(os.path.join(dw__image_folder_path, f"a647_{current_field_id:03d}.tif")).max(0)

labels = tifffile.imread(os.path.join(mask_folder_path, f"a647_{current_field_id:03d}.dilated_labels.tiff")).reshape(raw_max_z_proj.shape)

field_raw_dots = ref_raw.loc[ref_raw["series_id"] == current_field_id, :].sort_values("Value2", ascending=False)
field_dw__dots = ref__dw.loc[ref__dw["series_id"] == current_field_id, :].sort_values("Value2", ascending=False)

selected_raw_dots = field_raw_dots.reset_index(drop=True)
selected_dw__dots = field_dw__dots.reset_index(drop=True)

fig3, ax = plt.subplots(figsize=(20, 10), ncols=2, constrained_layout=True)
fig3.suptitle(f"Field #{current_field_id}")
print(" > Plotting raw")
ax[0].set_title(f"60x_raw (n.dots={selected_raw_dots.shape[0]})")
ax[0].imshow(
    raw_max_z_proj, cmap=plt.get_cmap("gray"), interpolation="none",
    vmin=raw_max_z_proj.min(), vmax=raw_max_z_proj.max(),
    resample=False, filternorm=False)
ax[0].scatter(
    x=selected_raw_dots["y"].values,
    y=selected_raw_dots["x"].values,
    s=30, facecolors='none', edgecolors='r', linewidth=.5)
print(" > Plotting dw")
ax[1].set_title(f"60x_dw (n.dots={selected_dw__dots.shape[0]})")
ax[1].imshow(
    dw__max_z_proj, cmap=plt.get_cmap("gray"), interpolation="none",
    vmin=dw__max_z_proj.min()*1.5, vmax=dw__max_z_proj.max()*.5,
    resample=False, filternorm=False)
ax[1].scatter(
    x=selected_dw__dots["y"].values,
    y=selected_dw__dots["x"].values,
    s=30, facecolors='none', edgecolors='r', linewidth=.5)
print(" > Plotting contours")
for lid in tqdm(range(1, labels.max()), desc="nucleus"):
    contours = measure.find_contours(labels == lid, 0.8)
    for contour in contours:
        ax[0].scatter(x=contour[:,1], y=contour[:,0], c="yellow", s=.005)
        ax[1].scatter(x=contour[:,1], y=contour[:,0], c="yellow", s=.005)
plt.close(fig3)
print(" > Exporting")
fig3.savefig(f"overlay_{current_field_id:03d}.60x.png", bbox_inches='tight')
print(" ! DONE")

Field #1


nucleus:   0%|          | 0/180 [00:00<?, ?it/s]

 > Plotting raw
 > Plotting dw
 > Plotting contours


nucleus: 100%|██████████| 180/180 [00:12<00:00, 14.32it/s]


 > Exporting
 ! DONE
